访问数据是使用各类工具所必需的第一步。将重点关注使用pandas进行数据输入和输出，尽管其他库中有许多工具可帮助读取和写入各种格式的数据。

输入和输出通常有以下几种类型：读取文本文件及硬盘上其他更高效的格式文件、从数据库载入数据、与网络资源进行交互（比如Web API）。

In [1]:
# 引入默认库
import pandas as pd
import numpy as np

## 6.1　文本格式数据的读写
将表格型数据读取为DataFrame对象是pandas的重要特性。下表总结了部分实现该功能的函数，`read_csv`和`read_table`可能是后期使用最多的函数。

|      函数      |                            描述                             |
| -------------- | ---------------------------------------------------------- |
| read_csv       | 从文件、URL或文件型对象读取分隔好的数据,逗号是默认分隔符          |
| read_table(将弃用)| 从文件、URL或文件型对象读取分隔好的数据,制表符('\\t')是默认分隔符 |
| read_fwf       | 从特定宽度格式的文件中读取数据(无分隔符)                        |
| read_clipboard | read_table的剪贴板版本,在将表格从Web页面上转换成数据时有用      |
| read_excel     | 从 Excel的xls或xlsx文件中读取表格数据                         |
| read_hdf       | 读取用 pandas存储的HDF5文件                                  |
| read_html      | 从HTML文件中读取所有表格数据                                  |
| read_json      | 从JSON( JavaScript Object Notation)字符串中读取数据          |
| read_msgpack   | 读取 MessagePack二进制格式的 pandas数据                       |
| read_pickle    | 读取以 Python pickle格式存储的任意对象                        |
| read_sas       | 读取存储在SAS系统中定制存储格式的SAS数据集                     |
| read_sql       | 将SQL查询的结果(使用 SQLAlchemy)读取为 pandas的 DataFrame     |
| read_stata     | 读取 Stata格式的数据集                                       |
| read_feather   | 读取 Feather二进制格式                                       |

下面会解释这些函数将文本数据转换为DataFrame的机制，这些函数的可选参数主要有以下几种类型。

1. 索引:

可以将一或多个列作为返回的DataFrame，从文件或用户处获得列名，或者没有列名。

2. 类型推断和数据转换

包括用户自定义的值转换和自定义的缺失值符号列表。

3. 日期时间解析

包括组合功能，也包括将分散在多个列上的日期和时间信息组合成结果中的单个列。

4. 迭代

支持对大型文件的分块迭代。

5. 未清洗数据问题

跳过行、页脚、注释以及其他次要数据，比如使用逗号分隔千位的数字。

由于现实世界中的数据非常混乱，随着时间推移，一些数据加载函数（尤其是`read_csv`）的可选参数变得非常复杂。面对大量不同的参数，感到困难是正常的（`read_csv`已经有超过50个参数了）。pandas的在线文档有大量示例展示这些参数是如何工作的。如果在读取某个文件时遇到了困难，在文档中可能会有相似的示例帮助找到正确的参数。

一些数据载入函数，比如`pandas.read_csv`，会进行类型推断，因为列的数据类型并不是数据格式的一部分。那就意味着你不必指定哪一列是数值、整数、布尔值或字符串。其他的数据格式，比如HDF5、Feather和msgpack在格式中已经存储了数据类型。

处理日期和其他自定义类型可能需要额外的努力。从一个小型的逗号分隔文本文件（CSV）开始：

In [2]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

> 这里使用了Unix shell的`cat`命令来打印文件的原始内容。如果操作系统是Windows，可以使用`type`来代替`cat`达到同样的效果。

由于这个文件是逗号分隔的，可以使用read_csv将它读入一个DataFrame：

In [3]:
df = pd.read_csv('examples/ex1.csv')

df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


也可以使用read_table，并指定分隔符：

In [4]:
pd.read_table('examples/ex1.csv', sep=',')

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


有的文件并不包含表头行。考虑以下文件：

In [5]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

要读取该文件，需要选择一些选项。可以允许pandas自动分配默认列名，也可以自己指定列名：

In [6]:
pd.read_csv('examples/ex2.csv', header=None, )

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


假设想要message列成为返回DataFrame的索引，你可以指定位置4的列为索引，或将'message'传给参数index_col：

In [8]:
names = ['a', 'b', 'c', 'd', 'message']

pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


当想要从多个列中形成一个分层索引，需要传入一个包含列序号或列名的列表：

In [9]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16

In [10]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])

parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

在某些情况下，一张表的分隔符并不是固定的，使用空白或其他方式来分隔字段。考虑如下文本文件：

In [11]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

当字段是以多种不同数量的空格分开时，尽管可以手工处理，但在这些情况下也可以向`read_csv`传入一个正则表达式作为分隔符。在本例中，正则表达式为`\s+`，因此可以得到：

In [12]:
result = pd.read_csv('examples/ex3.txt', sep='\s+')

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
result.loc['aaa',:]

A   -0.264438
B   -1.026059
C   -0.619500
Name: aaa, dtype: float64

> 本例中，由于列名的数量比数据的列数少一个，因此`read_csv`推断第一列应当作为DataFrame的索引。

解析函数有很多附加参数帮助处理各种发生异常的文件格式（表6-2列举了一部分）。例如，可以使用`skiprows`来跳过第一行、第三行和第四行：

In [14]:
!cat examples/ex4.csv

pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 4])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,5,6,7.0,8.0,world
2,9,10,11.0,12.0,foo


`缺失值处理`是文件解析过程中一个重要且常常微妙的部分。通常情况下，缺失值要么不显示（空字符串），要么用一些标识值。默认情况下，pandas使用一些常见的标识，例如NA和NULL：

In [15]:
!cat examples/ex5.csv

result = pd.read_csv('examples/ex5.csv')

result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


`na_values`选项可以传入一个列表或一组字符串来处理缺失值：

In [17]:
# 将读取到的对应数据更改为np.nan
result = pd.read_csv('examples/ex5.csv', na_values=['2'],)

result

,something,a,b,c,d,message
0,one,1,NaN,3.0,4,NaN
1,two,5,6.0,NaN,8,world
2,three,9,10.0,11.0,12,foo


在字典中，每列可以指定不同的缺失值标识：

In [18]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


下表列举了`pandas.read_csv`和`pandas.read_table`中常用的选项。
> 最好的方法还是看帮助文档，其中还有具体示例。

|       参数       |                                                                                   描述                                                                                   |
| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| path            | 表明文件系统位置的字符串、URL或文件型对象                                                                                                                                     |
| sep或 delimiter | 用于分隔每行字段的字符序列或正则表达式                                                                                                                                        |
| header          | 用作列名的行号,默认是0(第一行),如果没有列名的话,应该为None                                                                                                                     |
| index_col       | 用作结果中行索引的列号或列名,可以是一个单一的名称/数字,也可以是一个分层索引                                                                                                        |
| names           | 结果的列名列表,和 \`header=None\`一起用                                                                                                                                    |
| skiprows        | 从文件开头处起,需要跳过的行数或行号列表                                                                                                                                       |
| na_values       | 需要用NA替换的值序列                                                                                                                                                       |
| comment         | 在行结尾处分隔注释的字符                                                                                                                                                    |
| parse_dates     | 尝试将数据解析为 datetime,默认是 False。<br>如果为True,将尝试解析所有的列。也可以指定列号或列名列表来进行解析。<br>如果列表的元素是元组或列表,将会把多个列组合在一起进行解析(例如日期/时间将拆分为两列) |
| keep date col   | 如果连接列到解析日期上,保留被连接的列,默认是 False                                                                                                                             |
| converters      | 包含列名称映射到函数的字典(例如{'foo':f}会把函数f应用到'foo'列)                                                                                                                |
| dayfirst        | 解析非明确日期时,按照国际格式处理(例如7/6/2012->June7,2012),默认为False                                                                                                       |
| date_parser     | 用于解析日期的函数                                                                                                                                                         |
| nrows           | 从文件开头处读入的行数                                                                                                                                                      |
| iterator        | 返回一个 TextParser对象,用于零散地读入文件                                                                                                                                   |
| chunksize       | 用于迭代的块大小                                                                                                                                                           |
| skip_footer     | 忽略文件尾部的行数                                                                                                                                                         |
| verbose         | 打印各种解析器输出的信息,比如位于非数值列中的缺失值数量                                                                                                                         |
| encoding        | Unicode文本编码(例如'utf-8′用于表示UTF-8编码的文本)                                                                                                                          |
| squeeze         | 如果解析数据只包含一列,返回一个 Series                                                                                                                                       |
| thousands       | 千位分隔符(例如',或'.)                                                                                                                                                     |

### 6.1.1　分块读入文本文件
当处理大型文件或找出正确的参数集来正确处理大文件时，可能需要读入文件的一个小片段或者按小块遍历文件。

在尝试大文件之前，可以先对pandas的显示设置进行调整，使之更为紧凑：

In [19]:
pd.options.display.max_rows = 10

现在可以得到：

In [20]:
result = pd.read_csv('examples/ex6.csv')

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


如果只想读取一小部分行（避免读取整个文件），可以指明`nrows`：

In [21]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


为了分块读入文件，可以指定`chunksize`作为每一块的行数：

In [22]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

chunker

`read_csv`返回的TextParser对象允许根据`chunksize`遍历文件。例如，可以遍历ex6.csv，并对`'key'`列聚合获得计数值：

In [23]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

可以得到：

In [24]:
tot

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
     ...  
5    157.0
2    152.0
0    151.0
9    150.0
1    146.0
Length: 36, dtype: float64

TextParser还具有`get_chunk`方法，允许按照任意大小读取数据块。

### 6.1.2　将数据写入文本格式
数据可以导出为分隔的形式。看下之前读取的CSV文件：

In [25]:
data = pd.read_csv('examples/ex5.csv')

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


使用DataFrame的`to_csv`方法，我们可以将数据导出为逗号分隔的文件：

In [26]:
data.to_csv('examples/out.csv')

!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


当然，其他的分隔符也是可以的（写入到`sys.stdout`时，控制台中打印的文本结果）：

In [27]:
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出时以`空字符串`出现。想要用其他标识值对缺失值进行标注：

In [28]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


如果没有其他选项被指定的话，行和列的标签都会被写入。不过二者也都可以禁止写入：

In [29]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


也可以仅写入列的子集，并且按照你选择的顺序写入：

In [30]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series也有`to_csv`方法：

In [31]:
dates = pd.date_range('5/1/2020', periods=7)

ts = pd.Series(np.arange(7), index=dates)

# Series使用to_csv需要加上`header`参数，否则会有警告。
ts.to_csv('examples/tseries.csv', header=False)

!cat "examples/tseries.csv"

2020-05-01,0
2020-05-02,1
2020-05-03,2
2020-05-04,3
2020-05-05,4
2020-05-06,5
2020-05-07,6


### 6.1.3　使用分隔格式
绝大多数的表型数据都可以使用函数`pandas.read_csv`从硬盘中读取。然而，在某些情况下，一些手动操作可能是必不可少的。接收一个带有一行或多行错误的文件并不少见，`read_csv`也无法解决这种情况。为了介绍一些基础工具，考虑如下的小型CSV文件：

In [32]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


对于任何带有单字符分隔符的文件，可以使用Python的内建csv模块。要使用它，需要将任一打开的文件或文件型对象传给`csv.reader`：

In [33]:
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

像遍历文件那样遍历reader会产生元组，元组的值为删除了引号的字符：

In [34]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


之后，就可以自行做一些必要处理，以将数据整理为需要的形式。首先将文件读取为行的列表：

In [35]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

然后，将数据拆分为列名行和数据行：

In [36]:
header, values = lines[0], lines[1:]

再然后，使用字典推导式和表达式`zip（*values）`生成一个包含数据列的字典，字典中行转置成列：

In [37]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV文件有多种不同风格。如需根据不同的分隔符、字符串引用约定或行终止符定义一种新的格式时，可以使用`csv.Dialect`定义一个简单的子类：

In [38]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

with open('examples/ex7.csv') as f:
    reader = csv.reader(f, dialect=my_dialect)
    for line in reader:
        print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


也可以不必定义子类，直接将CSV Dialect参数传入`csv.reader`的关键字参数：

In [39]:
with open('examples/ex7.csv') as f:
    reader = csv.reader(f, delimiter='|')
    for line in reader:
        print(line)

['a,"b","c"']
['1,"2","3"']
['1,"2","3"']


下表列出了`csv.Dialect`中的一些属性及其用途。


|       参数        |                                                                            描述                                                                            |
| ---------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------- |
| delimiter        | 一个用于分隔字段的字符,默认是','                                                                                                                              |
| lineterminator   | 行终止符,默认是'\\r\\n',读取器会忽略行终止符并识别跨平台行终止符                                                                                                  |
| quotechar        | 用在含有特殊字符字段中的引号,默认是'"'                                                                                                                         |
| quoting          | 引用惯例。选项包括csv.QUOTE\_ALL(引用所有的字段),<br>csv.QUOTE\_MINIMAL(只使用特殊字符,如分隔符),<br>csν.QUOTE\_NONNUMERIC和csv.QUOTE\_NONE(不引用)。<br>默认是 QUOTE_MINIMAL |
| skipinitialspace | 忽略每个分隔符后的空白,默认是False                                                                                                                            |
| doublequote      | 如何处理字段内部的引号。如果为True,则是双引号(完整细节和行为请参考在线文档)                                                                                         |
| escapechar       | 当引用设置为csv. QUOTE_NONE时用于转义分隔符的字符串,默认是禁用的                                                                                                 |

> 对于具有更复杂或固定的多字符分隔符的文件将无法使用csv模块。在此类情况下，你将不得不使用字符串的split方法或正则表达式方法`re.split`进行行拆分和其他清理工作。

需要手动写入被分隔的文件时，可以使用csv.writer。这个函数接收一个已经打开的可写入文件对象以及和csv.reader相同的CSV方言、格式选项：

In [40]:
with open('examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))
    
!cat "examples/mydata.csv"

one;two;three
1;2;3
4;5;6
7;8;9


### 6.1.4　JSON数据
`JSON`（JavaScript Object Notation的简写）已经成为Web浏览器和其他应用间通过HTTP请求发送数据的标准格式。它是一种比CSV等表格文本形式更为自由的数据形式。请看下面的例子：

In [41]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

JSON非常接近有效的Python代码，除了它的空值null和一些其他的细微差别（例如不允许列表末尾的逗号）之外。基本类型是对象（字典）、数组（列表）、字符串、数字、布尔值和空值。对象中的所有键都必须是字符串。有几个Python库用于读写JSON数据。在这里使用`json`库，它是内置在Python标准库中的。将JSON字符串转换为Python形式时，使用`json.loads`方法：

In [42]:
import json

result = json.loads(obj)

result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

另一方面，`json.dumps`可以将Python对象转换回JSON：

In [43]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

可以自行决定如何将JSON对象或对象列表转换为DataFrame或其他数据结构。比较方便的方式是将字典构成的列表（之前是JSON对象）传入DataFrame构造函数，并选出数据字段的子集：

In [44]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

siblings

,name,age
0,Scott,30
1,Katie,38


In [45]:
pd.DataFrame(result['siblings'])

,age,name,pets
0,30,Scott,"[Zeus, Zuko]"
1,38,Katie,"[Sixes, Stache, Cisco]"


`pandas.read_json`可以自动将JSON数据集按照指定次序转换为Series或DataFrame。例如：

In [46]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


`pandas.read_json`的默认选项是假设JSON数组中的每个对象是表里的一行：

In [47]:
data = pd.read_json('examples/example.json')

data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


如需了解读取、操作JSON数据（包括嵌套记录）的拓展示例，请参看后续章节的USDA食品数据库示例。

如果需要从pandas中将数据导出为JSON，一种办法是对Series和DataFrame使用`to_json`方法：

In [48]:
print(data.to_json())

print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### 6.1.5　XML和HTML：网络抓取
Python拥有很多可以对HTML和XML格式进行读取、写入数据的库，例如`lxml`（`http://lxml.de`）、`Beautiful Soup`和`html5lib`。尽管`lxml`是相对更快的库，但其他库可以更好地处理异常的HTML或XML文件。

pandas的内建函数`read_html`可以使用`lxml`和`Beautiful Soup`等库将HTML中的表自动解析为DataFrame对象。为了展示这个功能，从美国FDIC政府机构下载了显示银行倒闭数据的HTML文件（在pandas文档中使用）。首先，必须安装`read_html`所使用的附加库：
```sh
conda install lxml
pip3 install beautifulsoup4 html5lib
```
如果不用conda，`pip install lxml`也是可以的。

`pandas.read_html`函数有很多选项，但是默认情况下，会搜索并尝试解析所有包含在`<table>`标签中的表格型数据，返回的结果是DataFrame对象的列表：

In [49]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

len(tables)

1

In [50]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


因为failures有很多列，pandas在行内插入了换行符`\`。

此处可以着手一些数据清理和分析工作，比如计算每年银行倒闭的数量：

In [51]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### 6.1.5.1　使用lxml.objectify解析XML

`XML`（eXtensible Markup Language）是另一种常用的结构化数据格式，它使用元数据支持分层、嵌套数据。

之前，展示了`pandas.read_html`函数，它使用`lxml`或`Beautiful Soup`从HTML中解析数据。XML和HTML结构类似，但是XML更通用。这里将用一个例子展示如何使用lxml从更为通用的XML格式解析数据。

纽约大都会交通局（MTA）发布了一份关于其公交、火车服务（http://www.mta.info/developers/download.html ）的数据集。这里将关注性能数据，这部分数据包含在一个XML文件集合中。每个火车或公交服务都有一个不同的文件（例如Performance_MNR.xml代表地铁-北铁路），文件中以一系列XML记录的方式包含了按月的数据，如下面的例子：
```xml
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>
```
使用`lxml.objectify`，可以解析这个文件，并用`getroot`来获得对XML文件的根节点的引用：

In [52]:
from lxml import objectify

path = 'examples/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

`root.INDICATOR`返回一个生成器，可以产生每一个`<INDICATOR>`XML元素。对于每条记录，可以将标签名称的字典（如YTD_ACTUAL）填充为数据值（不包括几个标签）：

In [53]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

最后，将包含字典的列表转换为DataFrame：

In [54]:
perf = pd.DataFrame(data)

perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


XML数据可以比例子更复杂。每个标签也可以包含元数据。考虑一个HTML连接标签，也是有效的XML：

In [55]:
from io import StringIO
tag = '<a href="https://www.baidu.com">Baidu</a>'
root = objectify.parse(StringIO(tag)).getroot()

现在可以访问标签或链接文本中的任何字段（如href）：

In [56]:
print(root)
print(root.get('href'))
print(root.text)

Baidu
https://www.baidu.com
Baidu


## 6.2　二进制格式
使用Python内建的pickle序列化模块进行二进制格式操作是存储数据（也称为序列化）最高效、最方便的方式之一。pandas对象拥有一个to_pickle方法可以将数据以pickle格式写入硬盘：

In [57]:
frame = pd.read_csv('examples/ex1.csv')

frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [58]:
frame.to_pickle('examples/frame_pickle')

可以直接使用内建的pickle读取文件中“pickle化”的对象，或更方便地使用`pandas.read_pickle`做上述操作：

In [59]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


> pickle仅被推荐作为短期的存储格式。问题在于pickle很难确保格式的长期有效性；一个今天被pickle化的对象可能明天会因为库的新版本而无法反序列化。

pandas内建支持其他两个二进制格式：`HDFS`和`MessagePack`。推荐尝试不同的文件格式去看看它们的速度以及它们如何用于分析。pandas或NumPy其他的存储格式包括：

bcolz（http://bcolz.blosc.org/ ）

基于Blosc压缩库的可压缩列式二进制格式。

Feather（http://github.com/wesm/feather ）

R编程社区的Hadley Wickham（http://hadley.nz/ ）设计的跨语言列式文件格式。Feather使用Apache Arrow（http://arrow.apache.org ）列式存储器格式。

### 6.2.1　使用HDF5格式
HDF5是一个备受好评的文件格式，用于存储大量的科学数组数据。它以C库的形式提供，并且具有许多其他语言的接口，包括Java、Julia、MATLAB和Python。HDF5中的“HDF”代表分层数据格式。每个HDF5文件可以存储多个数据集并且支持元数据。与更简单的格式相比，HDF5支持多种压缩模式的即时压缩，使得重复模式的数据可以更高效地存储。HDF5适用于处理不适合在内存中存储的超大型数据，可以使高效读写大型数组的一小块。

尽管可以通过使用`PyTables`或`h5py`等库直接访问HDF5文件，但pandas提供了一个高阶的接口，可以简化Series和DataFrame的存储。`HDFStore`类像字典一样工作并处理低级别细节：
> 需要安装`PyTables`库，`pip3 install datasette-pytables`

In [60]:
frame = pd.DataFrame({'a': np.random.randn(100)})

store = pd.HDFStore('examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

包含在HDF5文件中的对象可以使用相同的字典型API进行检索：

In [61]:
store['obj1']

,a
0,0.542451
1,-0.467397
2,-1.732050
3,-0.861117
4,-0.652417
...,...
95,0.008996
96,0.240905
97,-0.388347
98,-0.306297


HDFStore支持两种存储模式，`'fixed'`和`'table'`。后者速度更慢，但支持一种特殊语法的查询操作：

In [62]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,1.293996
11,-0.200667
12,0.214138
13,0.957504
14,-0.864546
15,1.029112


In [63]:
store.close()

`put`是`store['obj2']=frame`方法的显式版本，但允许设置其他选项，如存储格式。

`pandas.read_hdf`函数是这些工具的快捷方法：

In [64]:
frame.to_hdf('mydata.h5', 'obj3', format='table')

pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,0.542451
1,-0.467397
2,-1.732050
3,-0.861117
4,-0.652417


> 如果处理存储在远程服务器上的数据时，比如Amazon S3或HDFS，使用其他专门为分布式存储而设计的二进制格式更为合适，比如Apache Parquet（http://parquet.apache.org ）。

如果是在本地处理大量数据，推荐尝试`PyTables`和`h5py`，看看它们是否符合需求。由于很多数据分析的困难在于I/O密集（而不是CPU密集），使用像HDF5这样的工具可以大大加速应用。

> HDF5并不是数据库，它是一种适合一次写入多次读取的数据集。尽管数据可以在任何时间添加到文件中，但如果多个写入者持续写入，文件可能会损坏。

### 6.2.2　读取Microsoft Excel文件
pandas也支持通过`ExcelFile`类或`pandas.read_excel`函数来读取存储在Excel 2003（或更高版本）文件中的表格型数据。这些工具内部是使用附加包`xlrd`和`openpyxl`来分别读取XLS和XLSX文件的。可能需要使用pip或conda手动安装这些工具。

使用ExcelFile时，通过将xls或xlsx的路径传入，生成一个实例：

In [65]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

存储在表中的数据可以通过`pandas.read_excel`读取到DataFrame中：

In [66]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果读取的是含有多个表的文件，生成ExcelFile更快，但也可以更简洁地将文件名传入`pandas.read_excel`：

In [67]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如需将pandas数据写入到Excel格式中，**必须**先生成一个ExcelWriter，然后使用pandas对象的to_excel方法将数据写入：

In [68]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.save()

也可以将文件路径传给to_excel，避免直接调用ExcelWriter：

In [69]:
frame.to_excel('examples/ex2.xlsx')

## 6.3　与Web API交互
很多网站都有公开API，通过JSON或其他格式提供数据服务。有多种方式可以利用Python来访问API；推荐的简单易用方式是使用`requests`包（http://docs.python-requests.org ）。

要获取GitHub上最新的30条关于pandas的问题，可以使用附加库`requests`发送一个HTTP GET请求：

In [70]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

Response（响应）对象的json方法将返回一个包含解析为本地Python对象的JSON的字典：

In [71]:
data = resp.json()

data[0]['title']

'ENH: fix arrow roundtrip for ExtensionDtypes in absence of pandas metadata'

data中的每个元素都是一个包含GitHub问题页面上的所有数据的字典（注释除外）。可以将data直接传给DataFrame，并提取感兴趣的字段：

In [72]:
issues = pd.DataFrame(data, columns=['number','title',
                                     'labels', 'state'])

issues

,number,title,labels,state
0,34275,ENH: fix arrow roundtrip for ExtensionDtypes i...,"[{'id': 685114413, 'node_id': 'MDU6TGFiZWw2ODU...",open
1,34274,BUG: small error in pandas/io/pytables.py,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,34273,QST:Different results of pd.merge if using on=...,"[{'id': 1954720290, 'node_id': 'MDU6TGFiZWwxOT...",open
3,34272,ENH: specify missing keys in KeyError when pas...,"[{'id': 42670965, 'node_id': 'MDU6TGFiZWw0MjY3...",open
4,34271,BUG: Same function calls on the same DataFrame...,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
...,...,...,...,...
25,34244,ENH: Support out-of-band pickling (protocol 5),"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
26,34243,CLN: remove get_base_alias,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
27,34241,CLN: Remove util.is_offset_object,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
28,34240,BUG: PeriodIndex.get_loc should raise KeyError,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


通过一些复杂操作，可以创建一些更高阶的接口来访问常用的Web API，以返回DataFrame对象以便于分析。

## 6.4　与数据库交互
在业务场景中，大部分数据并不是储存在文本或Excel文件中的。基于SQL的关系型数据库（例如SQL Server、PostgreSQL和MySQL）使用广泛，很多小众数据库也变得越发流行。数据库的选择通常取决于性能、数据完整性以及应用的可伸缩性需求。

从SQL中将数据读取为DataFrame是相当简单直接的，pandas有多个函数可以简化这个过程。作为例子，将使用Python内建的sqlite3驱动来生成一个SQLite数据库：

In [73]:
import sqlite3

query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20),
        c REAL,        d INTEGER
        );"""

con = sqlite3.connect('examples/mydata.sqlite')

con.execute(query)

con.commit()

再插入几行数据：

In [74]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

当从数据库的表中选择数据时，大部分Python的SQL驱动（PyODBC、psycopg2、MySQLdb、pymssql等）返回的是元组的列表：

In [75]:
cursor = con.execute('select * from test')

rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

可以将元组的列表传给DataFrame构造函数，但还需要包含在游标的`description`属性中的列名：

In [76]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [77]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


由于内容较多，肯定不想每次查询数据库都重复同样多的步骤。SQLAlchemy项目（http://www.sqlalchemy.org/ ）是一个流行的Python SQL工具包，抽象去除了SQL数据库之间的许多常见差异。pandas有一个`read_sql`函数允许从通用的SQLAlchemy连接中轻松地读取数据。这里，使用SQLAlchemy连接到相同的SQLite数据库，并从之前创建的表中读取数据：

In [78]:
In [135]: import sqlalchemy as sqla

In [136]: db = sqla.create_engine('sqlite:///examples/mydata.sqlite')

In [137]: pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
